<b>Создаем df's на базе наших Excel файлов</b>

In [158]:
import pandas as pd

activity = pd.read_excel('Предмет1_Посещаемость.xlsx')
scale = pd.read_excel('Предмет1-шкала.xlsx')
grades = pd.read_excel('Предмет 1 - Оценки.xlsx')
rating = pd.DataFrame()


<b>Разделение файла "Шкала" на баллы, баллы делим на 100</b>

In [159]:
points_scale = {
    'ЛК': float(scale.iloc[0, scale.columns.get_loc('Финально')]) / 100,
    'Посещаемость': float(scale.iloc[1, scale.columns.get_loc('Финально')]) / 100,
    'ЛР': float(scale.iloc[2, scale.columns.get_loc('Финально')]) / 100,
    'ПЗ': float(scale.iloc[3, scale.columns.get_loc('Финально')]) / 100,
    'КР': float(scale.iloc[4, scale.columns.get_loc('Финально')]) / 100,
    'ДЗ': float(scale.iloc[5, scale.columns.get_loc('Финально')]) / 100
}

points_scale

{'ЛК': 0.1, 'Посещаемость': 0.15, 'ЛР': 0.3, 'ПЗ': 0.25, 'КР': 0.1, 'ДЗ': 0.1}

<b>Преобразование колонок, с которыми мы будем работать, в тип float</b>

In [160]:
rating['Фамилия'] = activity['Фамилия']
rating['Сумма баллов'] = 0
activity['Процент'] = activity['Процент'].str.replace(',', '.').astype(float)

for columns, series in grades.items():
    if 'Задание' in columns:
        grades[columns] = pd.to_numeric(series, errors='coerce').astype(float)

<b>Подсчет баллов за посещаемость</b>

In [161]:
for columns, series in activity.items():
    if columns == 'Процент':
        for i in range(len(series)):
            rating.at[i, 'Сумма баллов'] += points_scale['Посещаемость'] * series[i]

rating


,Фамилия,Сумма баллов
0,Антонова,7.50
1,Блинов,8.25
2,Винокурова,10.50
3,Владимиров,6.00
4,Гордеева,15.00
5,Гусева,15.00
6,Демидов,0.00
7,Денисов,15.00
8,Егоров,15.00
9,Журавлева,15.00


<b>Создается словарь, чтобы подсчитать общее количество занятий</b>

In [162]:
value_of_sub = {'Лк': 0.0, 'Лр': 0.0, 'Пз': 0.0, 'Дз': 0.0, 'Кр': 0.0}

for columns, series in grades.items():
    if 'Лк' in columns:
       value_of_sub['Лк'] += 1
    elif 'ЛР' in columns:
        value_of_sub['Лр'] += 1
    elif 'Пз' in columns and 'КР' not in columns:
        value_of_sub['Пз'] += 1
    elif 'ДЗ' or 'Дз' in columns:
        value_of_sub['Дз'] += 1
    elif 'КР' in columns:
        value_of_sub['Кр'] += 1

value_of_sub

{'Лк': 9.0, 'Лр': 8.0, 'Пз': 22.0, 'Дз': 25.0, 'Кр': 0.0}

<b>Подсчет баллов за присланные лк, лр, пз и т.д.</b>

In [163]:
for columns, series in grades.items():
    if 'Лк' in columns:
        for i in range(len(series)):
            if pd.notna(series[i]):
                rating.at[i, 'Сумма баллов'] += (points_scale['ЛК'] / value_of_sub['Лк']) * series[i]
    elif 'ЛР' in columns:
        for i in range(len(series)):
            if pd.notna(series[i]):
                rating.at[i, 'Сумма баллов'] += (points_scale['ЛР'] / value_of_sub['Лр']) * series[i]
    elif 'Пз' in columns and 'КР' not in columns:
        for i in range(len(series)):
            if pd.notna(series[i]):
                rating.at[i, 'Сумма баллов'] += (points_scale['ПЗ'] / value_of_sub['Пз']) * series[i]
    elif 'ДЗ' or 'Дз' in columns:
        for i in range(len(series)):
            if pd.notna(series[i]):
                rating.at[i, 'Сумма баллов'] += (points_scale['ДЗ'] / value_of_sub['Дз']) * series[i]
    elif 'КР' in columns:
        for i in range(len(series)):
            if pd.notna(series[i]):
                rating.at[i, 'Сумма баллов'] += (points_scale['КР'] / value_of_sub['Кр']) * series[i]

sorted_rating = rating.sort_values(by='Сумма баллов', ascending=False)
sorted_rating = sorted_rating.reset_index(drop=True)
sorted_rating

,Фамилия,Сумма баллов
0,Марков,21.661616
1,Денисов,21.661616
2,Харитонов,20.580808
3,Гусева,20.467172
4,Скворцова,20.412879
5,Егоров,20.247475
6,Павлов,20.078283
7,Фирсова,19.910354
8,Кузьмина,19.568182
9,Мельникова,19.354798


<b>Выгрузка данных в Excel</b>

In [164]:
sorted_rating.to_excel('Рейтинг.xlsx', index=False)